In [5]:
            # West: rows i = 1..Nx-1 use face De[i-1,:] → vectorized:
            diag_w[1:, :]  = - th * dt * De[:,    :] / Cnp1[1:, :]
            # East: rows i = 0..Nx-2 use face De[i,:]
            diag_e[:-1, :] = - th * dt * De[:,    :] / Cnp1[:-1, :]
            # South: rows j = 1..Ny-1 use face Dn[:, j-1]
            diag_s[:, 1:]  = - th * dt * Dn[:, :] / Cnp1[:, 1:]
            # North: rows j = 0..Ny-2 use face Dn[:, j]
            diag_n[:, :-1] = - th * dt * Dn[:,    :] / Cnp1[:, :-1]

NameError: name 'th' is not defined